In [1]:
import pandas as pd
from sacremoses import MosesTokenizer
import string

In [2]:
mtk = MosesTokenizer()

In [13]:
train = pd.read_csv('train_ds.csv')
val = pd.read_csv('val_ds.csv')
test = pd.read_csv('test.csv')
train

,review,rating
0,Ga disappointed neat products .. Meletot Hilsn...,1
1,"Rdtanya replace broken glass, broken chargernya",1
2,Nyesel bngt dsni shopping antecedent photo mes...,1
3,Sent a light blue suit goods ga want a refund,1
4,Pendants came with dents and scratches on its ...,1
...,...,...
106881,Thanks you very satisfactory,5
106882,Excellent quality Excellent quality Very acco...,5
106883,Excellent product quality delivery speed is ve...,5
106884,thanks gan,5


In [4]:
train['review_clean'] = train['review'].str.lower()
train.sample(5)

,review,rating,review_clean
59165,But I have a good-sized sewing white. Lest the...,4,but i have a good-sized sewing white. lest the...
37922,7 ordered all bars Box 1 come rods are welded ...,3,7 ordered all bars box 1 come rods are welded ...
99141,Very accommodating seller havent tried the pr...,5,very accommodating seller havent tried the pr...
74869,"Kak good material, good jg response. Because q...",4,"kak good material, good jg response. because q..."
75517,Thank you so much sis .. hopefully continue to...,4,thank you so much sis .. hopefully continue to...


In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

- Punctuations that likely won't impact the sentiment `+*`
- Remove hashtag `#[a-z]+`
- Remove mention `@[a-z]+`
- Remove hyperlinks `http.+`


In [6]:
train[train['review_clean'].str.contains(r"\%")]

,review,rating,review_clean
1260,Items will not order again the same picture. ...,1,items will not order again the same picture. ...
1277,"Dishonest seller, the seller was promised the ...",1,"dishonest seller, the seller was promised the ..."
1383,The actual powerbank itself is defective. It g...,1,the actual powerbank itself is defective. it g...
1578,Earphonenya sure 100% ori? Products from Pampa...,1,earphonenya sure 100% ori? products from pampa...
1752,Material type but not cotton jersey 2xl as rep...,1,material type but not cotton jersey 2xl as rep...
...,...,...,...
105964,"Well, very good value for money, like buy 1 ge...",5,"well, very good value for money, like buy 1 ge..."
105987,"use good products, only 50% battery life but a...",5,"use good products, only 50% battery life but a..."
106011,[QC] THE NAP do not forget - today (09/20) 20%...,5,[qc] the nap do not forget - today (09/20) 20%...
106214,UDH Pketnny smpe lmyan cepet .. Tp syg earring...,5,udh pketnny smpe lmyan cepet .. tp syg earring...


In [7]:
train['review_clean'] = train['review_clean'].str.replace(r'’', '\'')
train['review_clean'] = train['review_clean'].str.replace(r'#[a-z]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'@[a-z]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'[\+\*=@#<>\(\)\[\]\^_]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'[,!?;.:]+', '. ')
train

,review,rating,review_clean
0,Ga disappointed neat products .. Meletot Hilsn...,1,ga disappointed neat products . meletot hilsn...
1,"Rdtanya replace broken glass, broken chargernya",1,rdtanya replace broken glass. broken chargernya
2,Nyesel bngt dsni shopping antecedent photo mes...,1,nyesel bngt dsni shopping antecedent photo mes...
3,Sent a light blue suit goods ga want a refund,1,sent a light blue suit goods ga want a refund
4,Pendants came with dents and scratches on its ...,1,pendants came with dents and scratches on its ...
...,...,...,...
106881,Thanks you very satisfactory,5,thanks you very satisfactory
106882,Excellent quality Excellent quality Very acco...,5,excellent quality excellent quality very acco...
106883,Excellent product quality delivery speed is ve...,5,excellent product quality delivery speed is ve...
106884,thanks gan,5,thanks gan


In [8]:
train.to_csv('train_clean.csv', index=False)

In [14]:
def clean(series):
    series = series.str.lower()

    series = series.str.replace(r'’', '\'')
    series = series.str.replace(r'#[a-z]+', ' ')
    series = series.str.replace(r'@[a-z]+', ' ')
    series = series.str.replace(r'[\+\*=@#<>\(\)\[\]\^_]+', ' ')
    series = series.str.replace(r'[,!?;.:]+', '. ')

    return series

val['review_clean'] = clean(val['review'])
test['review_clean'] = clean(test['review'])
val

,review,rating,review_clean
0,Order black. But until the color of khakis. Or...,1,order black. but until the color of khakis. ...
1,Response sellers alike absolutely nothing. ver...,1,response sellers alike absolutely nothing. ve...
2,Keep it in the bag rice weevil,1,keep it in the bag rice weevil
3,"Ga receipt of goods, confirmed to be from the ...",1,ga receipt of goods. confirmed to be from the...
4,goods bagussssssssssssssssssssssssssssssssssss...,1,goods bagussssssssssssssssssssssssssssssssssss...
...,...,...,...
9995,Original product. The price of the best parts....,5,original product. the price of the best parts...
9996,"CPET bgt his delivery, packing oke bgt, safe p...",5,cpet bgt his delivery. packing oke bgt. safe...
9997,Thank you for your boss at home really texture...,5,thank you for your boss at home really texture...
9998,Thank you seller....products delivered in good...,5,thank you seller. products delivered in good c...


In [15]:
val.to_csv('val_clean.csv', index=False)
test.to_csv('test_clean.csv', index=False)